In [6]:
!pip install --user --trusted-host pypi-registry.supplementary-services.svc.cluster.local --index http://pypi-registry.supplementary-services.svc.cluster.local:8080/ clickhouse-driver

Looking in indexes: http://pypi-registry.supplementary-services.svc.cluster.local:8080/
     |████████████████████████████████| 570 kB 3.3 MB/s eta 0:00:01


In [98]:
from clickhouse_driver import Client

In [99]:
client1 = Client('clickhouse-1')


In [100]:
client1.execute('show databases')

[('_temporary_and_external_tables',),
 ('bd',),
 ('default',),
 ('system',),
 ('task',),
 ('vk',)]

In [4]:
client1.execute('create database task on cluster kube_clickhouse_cluster')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [165]:
client1.execute('use task')

[]

In [6]:
client1.execute('show databases')

[('_temporary_and_external_tables',),
 ('default',),
 ('system',),
 ('task',),
 ('vk',)]

In [40]:
client1.execute('GRANT ALL ON *.* TO task3_PK WITH GRANT OPTION on cluster kube_clickhouse_cluster;')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [10]:
client1.execute('create role solver on CLUSTER kube_clickhouse_cluster ;')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [10]:
client1.execute('show roles;')

[('solver',)]

### Settings for user creation has been granted in user.xml files.

In [13]:
client1.execute("CREATE USER IF NOT EXISTS task3_PK ON CLUSTER kube_clickhouse_cluster \
IDENTIFIED WITH plaintext_password BY '12345678' default ROLE solver")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [14]:
client1.execute('show users')

[('default',), ('task3_PK',)]

## Likes 

In [15]:
# observe the data in likes.json 
!head -3 ~/shared-data/clickhouse_data/likes.json

{"item_type":"post","owner_id":"-174416405","item_id":"1083","liker_id":"72689","ctime":"0000-00-00 00:00:00","like_date":"0000-00-00 00:00:00","post_date":"2019-03-15 07:34:00"}
{"item_type":"post","owner_id":"-174416405","item_id":"1103","liker_id":"72689","ctime":"0000-00-00 00:00:00","like_date":"0000-00-00 00:00:00","post_date":"2019-03-17 08:05:00"}
{"item_type":"post","owner_id":"-170321944","item_id":"328776","liker_id":"72689","ctime":"0000-00-00 00:00:00","like_date":"0000-00-00 00:00:00","post_date":"2019-03-11 23:50:00"}


In [7]:
# create table likes

client1.execute('CREATE TABLE IF NOT EXISTS task.likes ON CLUSTER kube_clickhouse_cluster\
(\
    item_type String,\
    owner_id Int64,\
    item_id Int64,\
    liker_id Int64,\
    ctime DateTime,\
    like_date DateTime,\
    post_date DateTime\
)\
ENGINE = MergeTree()\
PARTITION BY toYear(post_date)\
ORDER BY (liker_id, owner_id, item_id);')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [8]:
# table created using Distributed DDL Queries.
client1.execute('CREATE TABLE task.ddl_likes ON CLUSTER kube_clickhouse_cluster AS task.likes ENGINE = Distributed(kube_clickhouse_cluster, task, likes, xxHash64(owner_id));')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [9]:
# table created for kafka engine to import external data.
client1.execute("CREATE TABLE IF NOT EXISTS task.likes_kafka ON CLUSTER kube_clickhouse_cluster\
(\
    item_type String,\
    owner_id Int64,\
    item_id Int64,\
    liker_id Int64,\
    ctime DateTime,\
    like_date DateTime,\
    post_date DateTime\
)\
ENGINE = Kafka('kafka-svc:9092', 'task_likes', 'Cgroup') SETTINGS kafka_format = 'JSONEachRow', kafka_row_delimiter = '\n';")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [10]:
# creating mv to transfer data between kafka and ddl table.
client1.execute('CREATE MATERIALIZED VIEW task.mv_kafka_likes ON CLUSTER kube_clickhouse_cluster TO task.ddl_likes AS SELECT * FROM task.likes_kafka;')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [ ]:
# inserting json data into likeskafka table.
# opt/kafka_2.13-2.7.0/bin/kafka-console-producer.sh --topic task_likes --bootstrap-server kafka-svc:9092 < shared-data/clickhouse_data/likes.json

In [ ]:
# Creating replicating likes table
# client1.execute("CREATE TABLE IF NOT EXISTS task3.rep_likes ON CLUSTER kube_clickhouse_cluster\
# (\
#     item_type String,\
#     owner_id Int64,\
#     item_id Int64,\
#     liker_id Int64,\
#     ctime DateTime,\
#     like_date DateTime,\
#     post_date DateTime\
# )\
# ENGINE = ReplicatedMergeTree('/replicated _likes', 'rep_likes')\
# PARTITION BY toYear(post_date)\
# ORDER BY (liker_id, owner_id, item_id);")


In [11]:
client1.execute('select count() from task.likes')

[(0,)]

In [12]:
client1.execute('select count() from task.ddl_likes')

[(0,)]

In [24]:
client1.execute("SELECT \
    owner_id, \
    count() AS likes_count \
FROM task.likes \
GROUP BY owner_id ORDER BY likes_count desc limit 5;")

[(-12382740, 153094),
 (-28950133, 74455),
 (-31836774, 60841),
 (-36164349, 59592),
 (-6136139, 57391)]

In [ ]:
# creating materlized view 1 for likes table.
client1.execute("CREATE MATERIALIZED VIEW task.mv1_likes ON CLUSTER kube_clickhouse_cluster \
ENGINE = AggregatingMergeTree() \
ORDER BY (likes_count) \
POPULATE \
AS SELECT \
    owner_id, \
    countState() AS likes_count \
FROM task.likes \
GROUP BY owner_id ORDER BY likes_count desc;")

In [28]:
client1.execute("CREATE TABLE task.ddl_mv1_likes ON CLUSTER kube_clickhouse_cluster AS task.mv1_likes ENGINE = Distributed(kube_clickhouse_cluster, task, mv1_likes)")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [138]:
client1.execute("SELECT \
    liker_id, \
    count() AS likes_count \
FROM task.likes where liker_id is not null \
GROUP BY liker_id order by likes_count desc limit 5;")

[(141042511, 35110),
 (268735255, 31082),
 (132131032, 27685),
 (151432057, 26635),
 (1881674, 23499)]

In [36]:
client1.execute("CREATE TABLE task.ddl_mv2_likes ON CLUSTER kube_clickhouse_cluster AS task.mv2_likes ENGINE = Distributed(kube_clickhouse_cluster, task, mv2_likes);")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [142]:
client1.execute("select * from task.mv2_likes order by likes_count desc limit 5")

[(141042511, 35110),
 (268735255, 31082),
 (132131032, 27685),
 (151432057, 26635),
 (1881674, 23499)]

In [143]:
client1.execute("select * from task.ddl_mv2_likes order by likes_count desc limit 5")

[(30052656, 55078),
 (132131032, 50095),
 (141042511, 49586),
 (59400881, 39197),
 (133662976, 38612)]

## followers

In [27]:
!head -3 ~/shared-data/clickhouse_data/followers.json

{"profile_id":"328582","follower_id":"4652928","ctime":"2020-03-07 10:09:50"}
{"profile_id":"328582","follower_id":"5638400","ctime":"2020-03-07 10:09:50"}
{"profile_id":"328582","follower_id":"5800954","ctime":"2020-03-07 10:09:50"}


In [33]:
client1.execute("CREATE TABLE IF NOT EXISTS task.followers ON CLUSTER kube_clickhouse_cluster\
(\
    profile_id Int64,\
    follower_id Int64,\
    ctime DateTime) ENGINE = MergeTree()\
PARTITION BY toYear(ctime)\
ORDER BY (follower_id, profile_id);")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [35]:
client1.execute("CREATE TABLE task.ddl_followers ON CLUSTER kube_clickhouse_cluster AS task.followers ENGINE = Distributed(kube_clickhouse_cluster, task, followers, xxHash64(follower_id));")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [38]:
client1.execute("CREATE TABLE IF NOT EXISTS task.followers_kafka ON CLUSTER kube_clickhouse_cluster\
(\
    profile_id Int64,\
    follower_id Int64,\
    ctime DateTime) ENGINE = Kafka('kafka-svc:9092', 'task_followers', 'Cgroup') SETTINGS kafka_format = 'JSONEachRow', kafka_row_delimiter = '\n'")

[('clickhouse-2', 9000, 0, '', 2, 2),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [40]:
# creating mv to transfer data between kafka and ddl table.
client1.execute("CREATE MATERIALIZED VIEW task.mv_kafka_followers TO task.ddl_followers AS SELECT * FROM task.followers_kafka;")

[]

In [ ]:
# inserting json data into likeskafka table.
# opt/kafka_2.13-2.7.0/bin/kafka-console-producer.sh --topic task_followers --bootstrap-server kafka-svc:9092 < shared-data/clickhouse_data/followers.json

In [144]:
client1.execute("SELECT \
    profile_id, \
    count() AS follower_count \
FROM task.followers \
GROUP BY profile_id order by follower_count desc limit 5;")

[(5592362, 334252),
 (22851406, 200018),
 (16652378, 149531),
 (93808169, 93115),
 (82029701, 69525)]

In [41]:
client1.execute('drop table task.ddl_mv1_followers ON CLUSTER kube_clickhouse_cluster')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [42]:
# creating materlized view 1 for followers table.
client1.execute("CREATE MATERIALIZED VIEW task.mv1_followers ON CLUSTER kube_clickhouse_cluster \
ENGINE = AggregatingMergeTree() \
ORDER BY (follower_count) \
POPULATE \
AS SELECT \
    profile_id, \
    countState() AS follower_count \
FROM task.followers \
GROUP BY profile_id;")

[('clickhouse-2', 9000, 0, '', 2, 2),
 ('clickhouse-3', 9000, 0, '', 1, 1),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [43]:
client1.execute("CREATE TABLE task.ddl_mv1_followers ON CLUSTER kube_clickhouse_cluster AS task.mv1_followers ENGINE = Distributed(kube_clickhouse_cluster, task, mv1_followers);")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [43]:
# creating materlized view 2 for followers table.
client1.execute("CREATE MATERIALIZED VIEW task.mv2_followers ON CLUSTER kube_clickhouse_cluster \
ENGINE = AggregatingMergeTree() \
ORDER BY (followers_count) \
POPULATE \
AS SELECT \
    follower_id, \
    countState() AS followers_count \
FROM task.followers \
GROUP BY follower_id;")

[('clickhouse-2', 9000, 0, '', 2, 2),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [44]:
client1.execute("CREATE TABLE task.ddl_mv2_followers ON CLUSTER kube_clickhouse_cluster AS task.mv2_followers ENGINE = Distributed(kube_clickhouse_cluster, task, mv2_followers);")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

## friends

In [51]:
!head -3 ~/shared-data/clickhouse_data/friends.json

{"user_id":"6","friend_id":"197","ctime":"2020-03-03 18:25:31"}
{"user_id":"6","friend_id":"2675","ctime":"2020-03-03 18:25:31"}
{"user_id":"6","friend_id":"2807","ctime":"2020-03-03 18:25:31"}


In [81]:
client1.execute('drop table task.rep_friends ON CLUSTER kube_clickhouse_cluster')

[('clickhouse-2', 9000, 0, '', 2, 2),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [107]:
client1.execute("CREATE TABLE IF NOT EXISTS task.repFriends \
(\
    user_id Int64,\
    friend_id Int64,\
    ctime DateTime\
)\
ENGINE = ReplicatedMergeTree('/clickhouse/tables/{shard}/task/repFriends', '{replica}')\
PARTITION BY toYear(ctime)\
ORDER BY (friend_id, user_id);")

[]

In [103]:
client1.execute("CREATE TABLE IF NOT EXISTS task.friends_rep\
(\
    user_id Int64,\
    friend_id Int64,\
    ctime DateTime\
)\
ENGINE = ReplicatedMergeTree('/clickhouse/tables/01/friends_rep', 'friends_rep1')\
PARTITION BY toYear(ctime)\
ORDER BY (friend_id, user_id);")

[]

In [ ]:
client1.execute("CREATE TABLE IF NOT EXISTS task.friends_rep\
(\
    user_id Int64,\
    friend_id Int64,\
    ctime DateTime\
)\
ENGINE = ReplicatedMergeTree('/clickhouse/tables/02/friends_rep', 'friends_rep2')\
PARTITION BY toYear(ctime)\
ORDER BY (friend_id, user_id);")

In [ ]:
CREATE TABLE IF NOT EXISTS task.friends_rep\
(\
    user_id Int64,\
    friend_id Int64,\
    ctime DateTime\
)\
ENGINE = ReplicatedMergeTree('/clickhouse/tables/03/friends_rep', 'friends_rep3')\
PARTITION BY toYear(ctime)\
ORDER BY (friend_id, user_id);

In [ ]:
SELECT *
FROM system.replicas
WHERE table = 'friends_rep'
FORMAT Vertical

In [48]:
client1.execute("CREATE TABLE IF NOT EXISTS task.friends ON CLUSTER kube_clickhouse_cluster\
(\
    user_id Int64,\
    friend_id Int64,\
    ctime DateTime\
)\
ENGINE = ReplicatedMergeTree('/friends', 'friends')\
PARTITION BY toYear(ctime)\
ORDER BY (friend_id, user_id);")

[('clickhouse-1', 9000, 0, '', 2, 1),
 ('clickhouse-3', 9000, 0, '', 1, 1),
 ('clickhouse-2', 9000, 0, '', 0, 0)]

In [50]:
client1.execute("CREATE TABLE task.ddl_friends ON CLUSTER kube_clickhouse_cluster AS task.friends ENGINE = Distributed(kube_clickhouse_cluster, task, friends, xxHash64(user_id));")

[('clickhouse-1', 9000, 0, '', 2, 2),
 ('clickhouse-2', 9000, 0, '', 1, 1),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [52]:
client1.execute("CREATE TABLE IF NOT EXISTS task.friends_kafka ON CLUSTER kube_clickhouse_cluster\
(\
    user_id Int64,\
    friend_id Int64,\
    ctime DateTime) Kafka('kafka-svc:9092', 'task_friends', 'task') SETTINGS kafka_format = 'JSONEachRow', kafka_row_delimiter = '\n'")

[('clickhouse-1', 9000, 0, '', 2, 2),
 ('clickhouse-2', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [54]:
# creating mv to transfer data between kafka and ddl table.
client1.execute("CREATE MATERIALIZED VIEW task.mv_kafka_friends TO task.ddl_friends AS SELECT * FROM task.friends_kafka;")

[]

In [ ]:
# inserting json data into likeskafka table.
# opt/kafka_2.13-2.7.0/bin/kafka-console-producer.sh --topic task_friends --bootstrap-server kafka-svc:9092 < shared-data/clickhouse_data/friends.json

In [151]:
client1.execute("SELECT \
    user_id, \
    count() AS friends_count \
FROM task.friends \
GROUP BY user_id order by friends_count desc limit 5;")

[(295100189, 20336),
 (4230380, 18300),
 (248532383, 17936),
 (416658557, 13226),
 (22944210, 12754)]

In [48]:
client1.execute('drop table task.ddl_mv1_friends ON CLUSTER kube_clickhouse_cluster')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [47]:
# creating materlized view 1 for friends table.
client1.execute("CREATE MATERIALIZED VIEW task.mv1_friends ON CLUSTER kube_clickhouse_cluster \
ENGINE = AggregatingMergeTree() \
ORDER BY (friends_count) \
POPULATE \
AS SELECT \
    user_id, \
    countState() AS friends_count \
FROM task.friends \
GROUP BY user_id;")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [49]:
client1.execute("CREATE TABLE task.ddl_mv1_friends ON CLUSTER kube_clickhouse_cluster AS task.mv1_friends ENGINE = Distributed(kube_clickhouse_cluster, task, mv1_friends);")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [57]:
# creating materlized view 2 for friends table.
client1.execute("CREATE MATERIALIZED VIEW task.mv2_friends ON CLUSTER kube_clickhouse_cluster \
ENGINE = AggregatingMergeTree() \
ORDER BY (user_count) \
POPULATE \
AS SELECT \
    friend_id, \
    countState() AS user_count \
FROM task.friends \
GROUP BY friend_id;")

[('clickhouse-1', 9000, 0, '', 2, 2),
 ('clickhouse-2', 9000, 0, '', 1, 1),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [58]:
client1.execute("CREATE TABLE task.ddl_mv2_friends ON CLUSTER kube_clickhouse_cluster AS task.mv2_friends ENGINE = Distributed(kube_clickhouse_cluster, task, mv2_friends);")

[('clickhouse-2', 9000, 0, '', 2, 1),
 ('clickhouse-1', 9000, 0, '', 1, 1),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

## posts

In [59]:
!head -2 ~/shared-data/clickhouse_data/posts.json

{"ctime":"0000-00-00 00:00:00","date":"2012-03-13 00:00:00","post_id":"5","from_id":"78062","owner_id":"-36559458","comments_count":-1,"likes_count":2,"reposts_count":0,"views_count":0,"text":"","signed_by":"-1","post_type":"post","reposted_from_owner_id":null,"reposted_from_post_id":null,"geo":0,"geo_lat":null,"geo_lon":null,"geo_hash":null,"photo_attachments.owner_id":[],"photo_attachments.photo_id":[],"photo_attachments.size":[],"photo_attachments.url":[],"photo_attachments.geo_lat":[],"photo_attachments.geo_lon":[],"photo_attachments.geo_hash":[],"video_attachments.owner_id":[],"video_attachments.video_id":[],"video_attachments.views":[],"audio_attachments.owner_id":[],"audio_attachments.audio_id":[],"audio_attachments.artist":[],"audio_attachments.title":[],"doc_attachments.owner_id":[],"doc_attachments.doc_id":[],"doc_attachments.title":[],"doc_attachments.size":[],"doc_attachments.url":[],"doc_attachments.type":[],"link_attachments.title":[],"link_attachments.url":[],"page_attac

In [ ]:
client1.execute("CREATE TABLE task.posts ON CLUSTER kube_clickhouse_cluster\
(\
    ctime Nullable(DateTime),\
    date Nullable(DateTime),\
    post_id Int64,\
    from_id Nullable(String),\
    owner_id Nullable(String),\
    comments_count Nullable(Int64),\
    likes_count Nullable(Int64),\
    reposts_count Nullable(Int64),\
    views_count Nullable(Int64),\
    text Nullable(String),\
    signed_by Nullable(String),\
    post_type Nullable(String),\
    reposted_from_owner_id Nullable(String),\
    reposted_from_post_id Nullable(String),\
    geo Nullable(Int64),\
    geo_lat Nullable(Float64),\
    geo_lon Nullable(Float64),\
    geo_hash Nullable(String),\
    \
    photo_attachments Nested\
    (\
        owner_id Nullable(String),\
        photo_id Nullable(String),\
        size Nullable(String),\
        url Nullable(String),\
        geo_lat Nullable(Float64),\
        geo_lon Nullable(Float64),\
        geo_hash Nullable(String)\
    ),\
\
    video_attachments Nested\
    (\
        owner_id Nullable(String),\
        video_id Nullable(String),\
        views Nullable(Int64)    \
    ),\
\
    audio_attachments Nested\
    (\
        owner_id Nullable(String),\
        audio_id Nullable(String),\
        artist Nullable(String),\
        title Nullable(String)\
    ),\
    \
    doc_attachments Nested\
    (\
        owner_id Nullable(String),\
        doc_id Nullable(String),\
        title Nullable(String),\
        size Nullable(String),\
        url Nullable(String),\
        type Nullable(Int64)\
    ),\
    \
    link_attachments Nested\
    (\
        title Nullable(String),\
        url Nullable(String)\
    ),\
    \
    page_attachments Nested\
    (\
        group_id Nullable(String),\
        page_id Nullable(String)\
    ),\
    \
    sticker_attachments Nested\
    (\
        sticker_id Nullable(String)\
    ),\
    \
    photos_attachments_count Nullable(Int64),\
    videos_attachments_count Nullable(Int64),\
    audios_attachments_count Nullable(Int64),\
    docs_attachments_count Nullable(Int64),\
    links_attachments_count Nullable(Int64),\
    pages_attachments_count Nullable(Int64),\
    stickers_attachments_count Nullable(Int64)\
    \
)\
ENGINE = ReplicatedMergeTree('/posts', 'posts')\
ORDER BY (post_id);")

In [65]:
client1.execute("CREATE TABLE task.ddl_posts ON CLUSTER kube_clickhouse_cluster AS task.posts ENGINE = Distributed(kube_clickhouse_cluster, task, posts, xxHash64(post_id));")

[('clickhouse-2', 9000, 0, '', 2, 1),
 ('clickhouse-1', 9000, 0, '', 1, 1),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [67]:
client1.execute("CREATE TABLE task.posts_kafka ON CLUSTER kube_clickhouse_cluster\
(\
    ctime Nullable(DateTime),\
    date Nullable(DateTime),\
    post_id Int64,\
    from_id Nullable(String),\
    owner_id Nullable(String),\
    comments_count Nullable(Int64),\
    likes_count Nullable(Int64),\
    reposts_count Nullable(Int64),\
    views_count Nullable(Int64),\
    text Nullable(String),\
    signed_by Nullable(String),\
    post_type Nullable(String),\
    reposted_from_owner_id Nullable(String),\
    reposted_from_post_id Nullable(String),\
    geo Nullable(Int64),\
    geo_lat Nullable(Float64),\
    geo_lon Nullable(Float64),\
    geo_hash Nullable(String),\
    \
    photo_attachments Nested\
    (\
        owner_id Nullable(String),\
        photo_id Nullable(String),\
        size Nullable(String),\
        url Nullable(String),\
        geo_lat Nullable(Float64),\
        geo_lon Nullable(Float64),\
        geo_hash Nullable(String)\
    ),\
\
    video_attachments Nested\
    (\
        owner_id Nullable(String),\
        video_id Nullable(String),\
        views Nullable(Int64)    \
    ),\
\
    audio_attachments Nested\
    (\
        owner_id Nullable(String),\
        audio_id Nullable(String),\
        artist Nullable(String),\
        title Nullable(String)\
    ),\
    \
    doc_attachments Nested\
    (\
        owner_id Nullable(String),\
        doc_id Nullable(String),\
        title Nullable(String),\
        size Nullable(String),\
        url Nullable(String),\
        type Nullable(Int64)\
    ),\
    \
    link_attachments Nested\
    (\
        title Nullable(String),\
        url Nullable(String)\
    ),\
    \
    page_attachments Nested\
    (\
        group_id Nullable(String),\
        page_id Nullable(String)\
    ),\
    \
    sticker_attachments Nested\
    (\
        sticker_id Nullable(String)\
    ),\
    \
    photos_attachments_count Nullable(Int64),\
    videos_attachments_count Nullable(Int64),\
    audios_attachments_count Nullable(Int64),\
    docs_attachments_count Nullable(Int64),\
    links_attachments_count Nullable(Int64),\
    pages_attachments_count Nullable(Int64),\
    stickers_attachments_count Nullable(Int64)\
    \
) ENGINE = Kafka('kafka-svc:9092', 'task_posts', 'Cgroup') SETTINGS kafka_format = 'JSONEachRow', kafka_row_delimiter = '\n';")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [68]:
# creating mv to transfer data between kafka and ddl table.
client1.execute("CREATE MATERIALIZED VIEW task.mv_kafka_posts TO task.ddl_posts AS SELECT * FROM task.posts_kafka;")

[]

In [ ]:
# inserting json data into likeskafka table.
# opt/kafka_2.13-2.7.0/bin/kafka-console-producer.sh --topic task_posts --bootstrap-server kafka-svc:9092 < shared-data/clickhouse_data/posts.json

In [161]:
client1.execute("SELECT \
    post_id, \
    avg(views_count) AS likes_count_avg, \
    sum(likes_count) AS likes_count_sum FROM task.posts GROUP BY post_id order by likes_count_sum desc limit 5;")

[(87014, 12736.0, 12736),
 (76893, 11150.0, 11150),
 (83274, 10746.0, 10746),
 (56842, 3515.6666666666665, 10547),
 (86964, 10145.0, 10145)]

In [52]:
client1.execute('drop table task.ddl_mv1_posts ON CLUSTER kube_clickhouse_cluster')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [51]:
# creating materlized view 1 for posts table.
client1.execute("CREATE MATERIALIZED VIEW task.mv1_posts ON CLUSTER kube_clickhouse_cluster \
ENGINE = SummingMergeTree() \
ORDER BY (post_id) \
POPULATE \
AS SELECT \
    post_id, \
    avgState(likes_count) AS likes_count_avg, \
    sumState(likes_count) AS likes_count_sum FROM task.posts GROUP BY post_id;")

[('clickhouse-2', 9000, 0, '', 2, 1),
 ('clickhouse-3', 9000, 0, '', 1, 1),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [53]:
client1.execute("CREATE TABLE task.ddl_mv1_posts ON CLUSTER kube_clickhouse_cluster AS task.mv1_posts ENGINE = Distributed(kube_clickhouse_cluster, task, mv1_posts);")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [171]:
client1.execute("SELECT \
    post_id, \
    min(views_count) AS views_count_min, \
    max(views_count) AS views_count_max, \
    avg(views_count) AS views_count_avg, \
    sum(views_count) AS views_count_sum FROM task.posts WHERE (views_count > 0) GROUP BY post_id \
    order by views_count_sum desc limit 5;")

[(2780, 1, 253000, 8565.129032258064, 265519),
 (70581, 17, 253000, 126508.5, 253017),
 (70978, 235000, 235000, 235000.0, 235000),
 (71621, 190000, 190000, 190000.0, 190000),
 (70381, 11, 142000, 71005.5, 142011)]

In [ ]:
CREATE MATERIALIZED VIEW task.mv2_posts \
ENGINE = AggregatingMergeTree() \
ORDER BY (post_id) \
POPULATE \
AS SELECT \
    post_id, \
    minState(views_count) AS views_count_min, \
    maxState(views_count) AS views_count_max, \
    avgState(views_count) AS views_count_avg, \
    sumState(views_count) AS views_count_sum FROM task.posts WHERE (views_count > 0) GROUP BY post_id;

In [55]:
# creating materlized view 2 for posts table.
client1.execute("CREATE MATERIALIZED VIEW task.mv2_posts ON CLUSTER kube_clickhouse_cluster \
ENGINE = AggregatingMergeTree() \
ORDER BY (post_id) \
POPULATE \
AS SELECT \
    post_id, \
    minState(views_count) AS views_count_min, \
    maxState(views_count) AS views_count_max, \
    avgState(views_count) AS views_count_avg, \
    sumState(views_count) AS views_count_sum FROM task.posts WHERE (views_count > 0) GROUP BY post_id;")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [ ]:
CREATE TABLE task.ddl_mv2_posts AS task.mv2_posts ENGINE = Distributed(kube_clickhouse_cluster, task, mv2_posts);

In [177]:
client1.execute("select * from task.ddl_mv2_posts order by views_count_avg desc limit 5;")

[(71003, 710000, 710000, 710000.0, 710000),
 (71038, 529000, 529000, 529000.0, 529000),
 (70879, 329000, 329000, 329000.0, 329000),
 (70978, 235000, 235000, 235000.0, 235000),
 (71621, 190000, 190000, 190000.0, 190000)]

### user_profiles

In [69]:
!head -2 ~/shared-data/clickhouse_data/user_profiles.json

{"ctime":"2019-01-14 14:43:44","id":"15588946","first_name":"Karina","last_name":"Enokyan","screen_name":"id15588946","maiden_name":"","nickname":"","bdate":"20.1","birth_date":null,"sex":1,"deactivated":-1,"is_closed":null,"verified":0,"followers_count":null,"status":"http:\/\/vk.com\/pomosh_selene","city_id":1,"city_title":"Moscow","country_id":1,"country_title":"Russia","mobile_phone":null,"home_phone":null,"tv":null,"twitter":null,"livejournal":null,"facebook":null,"site":null,"skype":null,"instagram":null,"about":null,"activities":null,"books":null,"home_town":null,"interests":null,"movies":null,"music":null,"games":null,"quotes":null,"domain":"id15588946","personal_alcohol":null,"personal_inspired_by":null,"personal_langs":[],"personal_life_main":null,"personal_people_main":null,"personal_political":null,"personal_religion":null,"personal_smoking":null,"relation":null,"relation_partner_first_name":null,"relation_partner_id":null,"relation_partner_last_name":null,"photo_id":null,"

In [ ]:
client1.execute("CREATE TABLE task.user_profiles ON CLUSTER kube_clickhouse_cluster\
(\
    ctime DateTime,\
    id Int64,\
    first_name String,\
    last_name String,\
    screen_name Nullable(String),\
    maiden_name Nullable(String),\
    nickname Nullable(String),\
    bdate Nullable(String),\
    birth_date Nullable(UInt32),\
    sex Int16,\
    deactivated Int8,\
    is_closed Nullable(Int8),\
    verified Int8,\
    followers_count Nullable(Int64),\
    status Nullable(String),\
    city_id Int32,\
    city_title Nullable(String),\
    country_id Int32,\
    country_title Nullable(String),\
    mobile_phone Nullable(String),\
    home_phone Nullable(String),\
    tv Nullable(String),\
    twitter Nullable(String),\
    livejournal Nullable(String),\
    facebook Nullable(String),\
    site Nullable(String),\
    skype Nullable(String),\
    instagram Nullable(String),\
    about Nullable(String),\
    activities Nullable(String),\
    books Nullable(String),\
    home_town Nullable(String),\
    interests Nullable(String),\
    movies Nullable(String),\
    music Nullable(String),\
    games Nullable(String),\
    quotes Nullable(String),\
    domain Nullable(String),\
    personal_alcohol Nullable(Int8),\
    personal_inspired_by Nullable(String),\
    personal_langs Array(Nullable(String)),\
    personal_life_main Nullable(Int8),\
    personal_people_main Nullable(Int8),\
    personal_political Nullable(Int16),\
    personal_religion Nullable(String),\
    personal_smoking Nullable(Int8),\
    relation Nullable(Int8),\
    relation_partner_first_name Nullable(String),\
    relation_partner_id Nullable(Int64),\
    relation_partner_last_name Nullable(String),\
    photo_id Nullable(String),\
    photo_max_url Nullable(String),\
    crop_photo_album_id Nullable(Int64),\
    crop_photo_date Nullable(DateTime),\
    crop_photo_id Nullable(Int64),\
    crop_photo_lat Nullable(Float64),\
    crop_photo_long Nullable(Float64),\
    crop_photo_owner_id Nullable(Int64),\
    crop_photo_max Nullable(Int32),\
    crop_photo_max_url Nullable(String),\
    crop_photo_post_id Nullable(Int64),\
    crop_photo_text Nullable(String),\
    occupation_id Nullable(Int64),\
    occupation_name Nullable(String),\
    occupation_type Nullable(String),\
    education_form Nullable(Int8),\
    education_status Nullable(String),\
    faculty Nullable(Int32),\
    faculty_name Nullable(String),\
    graduation Nullable(UInt32),\
    university Nullable(Int32),\
    university_name Nullable(String),\
\
    relatives Nested\
    (\
        user_id Nullable(Int64),\
        name Nullable(String),\
        type Nullable(String)\
    ),\
\
    career Nested\
    (\
        company Nullable(String),\
        group_id Nullable(Int64),\
        city_id Nullable(Int32),\
        country_id Nullable(Int32),\
        position Nullable(String),\
        from Nullable(UInt32),\
        until Nullable(UInt32)\
    ),\
\
    schools Nested\
    (\
        city_id Nullable(Int32),\
        class Nullable(String),\
        country_id Nullable(Int32),\
        id Nullable(Int32),\
        name Nullable(String),\
        speciality Nullable(String),\
        type Nullable(UInt8),\
        type_str Nullable(String),\
        year_from Nullable(UInt16),\
        year_graduated Nullable(UInt16),\
        year_to Nullable(UInt16)\
    ),\
\
    universities Nested\
    (\
        chair Nullable(UInt32),\
        chair_name Nullable(String),\
        education_form Nullable(String),\
        education_status Nullable(String),\
        faculty Nullable(UInt32),\
        faculty_name Nullable(String),\
        graduation Nullable(UInt32),\
        id Nullable(Int32),\
        name Nullable(String)\
    )\
)\
ENGINE = ReplicatedMergeTree('/user_profiles', 'user_profiles')\
ORDER BY (id);")

In [83]:
client1.execute("CREATE TABLE task.ddl_user_profiles ON CLUSTER kube_clickhouse_cluster AS task.user_profiles ENGINE = Distributed(kube_clickhouse_cluster, task, user_profiles, xxHash64(id));")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [84]:
client1.execute("CREATE TABLE task.user_profiles_kafka ON CLUSTER kube_clickhouse_cluster\
(\
    ctime DateTime,\
    id Int64,\
    first_name String,\
    last_name String,\
    screen_name Nullable(String),\
    maiden_name Nullable(String),\
    nickname Nullable(String),\
    bdate Nullable(String),\
    birth_date Nullable(UInt32),\
    sex Int16,\
    deactivated Int8,\
    is_closed Nullable(Int8),\
    verified Int8,\
    followers_count Nullable(Int64),\
    status Nullable(String),\
    city_id Int32,\
    city_title Nullable(String),\
    country_id Int32,\
    country_title Nullable(String),\
    mobile_phone Nullable(String),\
    home_phone Nullable(String),\
    tv Nullable(String),\
    twitter Nullable(String),\
    livejournal Nullable(String),\
    facebook Nullable(String),\
    site Nullable(String),\
    skype Nullable(String),\
    instagram Nullable(String),\
    about Nullable(String),\
    activities Nullable(String),\
    books Nullable(String),\
    home_town Nullable(String),\
    interests Nullable(String),\
    movies Nullable(String),\
    music Nullable(String),\
    games Nullable(String),\
    quotes Nullable(String),\
    domain Nullable(String),\
    personal_alcohol Nullable(Int8),\
    personal_inspired_by Nullable(String),\
    personal_langs Array(Nullable(String)),\
    personal_life_main Nullable(Int8),\
    personal_people_main Nullable(Int8),\
    personal_political Nullable(Int16),\
    personal_religion Nullable(String),\
    personal_smoking Nullable(Int8),\
    relation Nullable(Int8),\
    relation_partner_first_name Nullable(String),\
    relation_partner_id Nullable(Int64),\
    relation_partner_last_name Nullable(String),\
    photo_id Nullable(String),\
    photo_max_url Nullable(String),\
    crop_photo_album_id Nullable(Int64),\
    crop_photo_date Nullable(DateTime),\
    crop_photo_id Nullable(Int64),\
    crop_photo_lat Nullable(Float64),\
    crop_photo_long Nullable(Float64),\
    crop_photo_owner_id Nullable(Int64),\
    crop_photo_max Nullable(Int32),\
    crop_photo_max_url Nullable(String),\
    crop_photo_post_id Nullable(Int64),\
    crop_photo_text Nullable(String),\
    occupation_id Nullable(Int64),\
    occupation_name Nullable(String),\
    occupation_type Nullable(String),\
    education_form Nullable(Int8),\
    education_status Nullable(String),\
    faculty Nullable(Int32),\
    faculty_name Nullable(String),\
    graduation Nullable(UInt32),\
    university Nullable(Int32),\
    university_name Nullable(String),\
\
    relatives Nested\
    (\
        user_id Nullable(Int64),\
        name Nullable(String),\
        type Nullable(String)\
    ),\
\
    career Nested\
    (\
        company Nullable(String),\
        group_id Nullable(Int64),\
        city_id Nullable(Int32),\
        country_id Nullable(Int32),\
        position Nullable(String),\
        from Nullable(UInt32),\
        until Nullable(UInt32)\
    ),\
\
    schools Nested\
    (\
        city_id Nullable(Int32),\
        class Nullable(String),\
        country_id Nullable(Int32),\
        id Nullable(Int32),\
        name Nullable(String),\
        speciality Nullable(String),\
        type Nullable(UInt8),\
        type_str Nullable(String),\
        year_from Nullable(UInt16),\
        year_graduated Nullable(UInt16),\
        year_to Nullable(UInt16)\
    ),\
\
    universities Nested\
    (\
        chair Nullable(UInt32),\
        chair_name Nullable(String),\
        education_form Nullable(String),\
        education_status Nullable(String),\
        faculty Nullable(UInt32),\
        faculty_name Nullable(String),\
        graduation Nullable(UInt32),\
        id Nullable(Int32),\
        name Nullable(String)\
    )\
)\
ENGINE = Kafka('kafka-svc:9092', 'task_user_profiles', 'Cgroup') SETTINGS kafka_format = 'JSONEachRow', kafka_row_delimiter = '\n';")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [85]:
# creating mv to transfer data between kafka and ddl table.
client1.execute("CREATE MATERIALIZED VIEW task.mv_kafka_user_profiles TO task.ddl_user_profiles AS SELECT * FROM task.user_profiles_kafka;")

[]

In [ ]:
# inserting json data into likeskafka table.
# opt/kafka_2.13-2.7.0/bin/kafka-console-producer.sh --topic task_user_profiles --bootstrap-server kafka-svc:9092 < shared-data/clickhouse_data/user_profiles.json

In [180]:
client1.execute("SELECT \
    country_title, \
    avg(followers_count) AS followers_count_avg, \
    sum(followers_count) AS followers_count_sum FROM \
    task.user_profiles WHERE (followers_count > 0) GROUP BY country_title \
    order by followers_count_sum desc limit 5;")

[('Russia', 482.4599941383353, 19753842),
 ('Россия', 470.7341567291312, 5526419),
 ('Ukraine', 526.7338690041393, 4326592),
 (None, 365.38378746594003, 2681917),
 ('Украина', 559.2058954890576, 1252062)]

In [60]:
client1.execute('drop table task.ddl_mv1_user_profiles ON CLUSTER kube_clickhouse_cluster')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [59]:
# creating materlized view 1 for posts table.
client1.execute("CREATE MATERIALIZED VIEW task.mv1_user_profiles ON CLUSTER kube_clickhouse_cluster \
ENGINE = SummingMergeTree() \
ORDER BY (country_id) \
POPULATE \
AS SELECT \
    country_id, country_title, \
    avgState(followers_count) AS followers_count_avg, \
    sumState(followers_count) AS followers_count_sum FROM \
    task.user_profiles WHERE (followers_count > 0) GROUP BY country_title, country_id;")

[('clickhouse-2', 9000, 0, '', 2, 1),
 ('clickhouse-3', 9000, 0, '', 1, 1),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [61]:
client1.execute("CREATE TABLE task.ddl_mv1_user_profiles ON CLUSTER kube_clickhouse_cluster AS task.mv1_user_profiles ENGINE = Distributed(kube_clickhouse_cluster, task, mv1_user_profiles);")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [188]:
client1.execute("SELECT \
    last_name, \
    min(followers_count) AS followers_count_min, \
    max(followers_count) AS followers_count_max, \
    avg(followers_count) AS followers_count_avg, \
    sum(followers_count) AS followers_count_sum FROM, \
    task.user_profiles WHERE (followers_count > 0) GROUP BY last_name order by followers_count_avg desc limit 5;")

[('Dobrodushnaya', 742231, 742231, 742231.0, 742231),
 ('Raytman', 315512, 315512, 315512.0, 315512),
 ('Debolskaya', 196114, 196114, 196114.0, 196114),
 ('Nabatchikova', 161697, 161697, 161697.0, 161697),
 ('Mirbi', 156644, 156644, 156644.0, 156644)]

In [66]:
client1.execute('drop table task.ddl_mv2_user_profiles ON CLUSTER kube_clickhouse_cluster')

[('clickhouse-3', 9000, 0, '', 2, 0),
 ('clickhouse-2', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [ ]:
CREATE MATERIALIZED VIEW task.mv2_user_profiles \
ENGINE = AggregatingMergeTree() \
ORDER BY (last_name) \
POPULATE \
AS SELECT \
    last_name, \
    minState(followers_count) AS followers_count_min, \
    maxState(followers_count) AS followers_count_max, \
    avgState(followers_count) AS followers_count_avg, \
    sumState(followers_count) AS followers_count_sum FROM \
    task.user_profiles WHERE (followers_count > 0) GROUP BY last_name;

In [64]:
# creating materlized view 2 for posts table.
client1.execute("CREATE MATERIALIZED VIEW task.mv2_user_profiles ON CLUSTER kube_clickhouse_cluster \
ENGINE = SummingMergeTree() \
ORDER BY (last_name) \
POPULATE \
AS SELECT \
    last_name, \
    minState(followers_count) AS followers_count_min, \
    maxState(followers_count) AS followers_count_max, \
    avgState(followers_count) AS followers_count_avg, \
    sumState(followers_count) AS followers_count_sum FROM \
    task.user_profiles WHERE (followers_count > 0) GROUP BY last_name;")

[('clickhouse-1', 9000, 0, '', 2, 2),
 ('clickhouse-2', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [67]:
client1.execute("CREATE TABLE task.ddl_mv2_user_profiles ON CLUSTER kube_clickhouse_cluster AS task.mv2_user_profiles ENGINE = Distributed(kube_clickhouse_cluster, task, mv2_user_profiles);")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-3', 9000, 0, '', 1, 0),
 ('clickhouse-1', 9000, 0, '', 0, 0)]

In [ ]:
CREATE TABLE task.ddl_mv2_user_profiles AS task.mv2_user_profiles ENGINE = Distributed(kube_clickhouse_cluster, task, mv2_user_profiles);

## Explantion about the work
1. Table create using mergetree likes, and followers.
2. Table created using replications are friends, posts and user_profiles. (zookeeper macro settings are fine in config.xml file)
3. Kafka engine is used to import externam data into the tables.
4. for each table 2 materlized views are build and the views are distributed. total materilized views(9).
5. All tables and materilized views are distributed.
6. User and Role are created. (Authentication settings have been set in the users.xml file)

#### All the quaries for table, user, role, materilized views, distributed tables, replications, and inserting data into tables using kafka engine(commented) are mentioned above.

### Sample quaries creating materlized views with joins and materlized views with joining two or more tables.

In [68]:
client1.execute("use task")

[]

In [69]:
# displaying all the tables and views created
client1.execute("show tables")

[('.inner.dist_mv_likesProfiles',),
 ('.inner.mv1_followers',),
 ('.inner.mv1_friends',),
 ('.inner.mv1_likes',),
 ('.inner.mv1_posts',),
 ('.inner.mv1_user_profiles',),
 ('.inner.mv2_likes',),
 ('.inner.mv2_posts',),
 ('.inner.mv2_user_profiles',),
 ('.inner.mv_likesProfiles',),
 ('.inner.mv_likes_profiles',),
 ('ddl_followers',),
 ('ddl_friends',),
 ('ddl_likes',),
 ('ddl_mv1_followers',),
 ('ddl_mv1_friends',),
 ('ddl_mv1_likes',),
 ('ddl_mv1_posts',),
 ('ddl_mv1_user_profiles',),
 ('ddl_mv2_likes',),
 ('ddl_mv2_posts',),
 ('ddl_mv2_user_profiles',),
 ('ddl_mv_likesProfiles',),
 ('ddl_mv_likes_profiles',),
 ('ddl_posts',),
 ('ddl_user_profiles',),
 ('dist_mv_likesProfiles',),
 ('followers',),
 ('followers_kafka',),
 ('friends',),
 ('friends_kafka',),
 ('likes',),
 ('likes_kafka',),
 ('mv1_followers',),
 ('mv1_friends',),
 ('mv1_likes',),
 ('mv1_posts',),
 ('mv1_user_profiles',),
 ('mv2_likes',),
 ('mv2_posts',),
 ('mv2_user_profiles',),
 ('mv_kafka_followers',),
 ('mv_kafka_friends'

In [195]:
# sample quary displaying the containts of likes table
client1.execute("select * from likes limit 3")

[('post',
  1666872,
  3040,
  1,
  datetime.datetime(2019, 2, 19, 16, 28),
  datetime.datetime(1970, 1, 1, 0, 0),
  datetime.datetime(1970, 1, 1, 0, 0)),
 ('post',
  1666872,
  4267,
  1,
  datetime.datetime(2019, 2, 19, 16, 28, 2),
  datetime.datetime(1970, 1, 1, 0, 0),
  datetime.datetime(1970, 1, 1, 0, 0)),
 ('post',
  1666872,
  6382,
  2,
  datetime.datetime(2019, 2, 19, 16, 28),
  datetime.datetime(1970, 1, 1, 0, 0),
  datetime.datetime(1970, 1, 1, 0, 0))]

In [204]:
client1.execute("SELECT liker_id, first_name, count() as cnt FROM \
( \
SELECT * FROM task.likes as l \
GLOBAL INNER JOIN task.user_profiles as p \
ON l.liker_id = p.id \
) \
GROUP BY liker_id, first_name order by cnt desc limit 5;")

[(268735255, 'Аня', 31082),
 (205266925, 'Lana', 20968),
 (39608712, 'Наташа', 16919),
 (174493123, 'Angela', 16358),
 (188817476, 'Марина', 16346)]

In [225]:
client1.execute("SELECT liker_id, first_name, count() as cnt FROM \
( \
SELECT * FROM task.ddl_likes as l \
GLOBAL INNER JOIN task.ddl_user_profiles as p \
ON l.liker_id = p.id \
) \
GROUP BY liker_id, first_name order by cnt desc limit 5;")

[(141042511, 'Анастасия', 119986),
 (132131032, 'Денис', 97031),
 (30052656, 'Валерия', 82405),
 (151432057, 'Valentina', 70073),
 (1881674, 'Аня', 63377)]

In [72]:
client1.execute('drop table task.ddl_mv_likesProfiles ON CLUSTER kube_clickhouse_cluster')

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [ ]:
CREATE MATERIALIZED VIEW task.mv_likesProfiles ON CLUSTER kube_clickhouse_cluster \
ENGINE = SummingMergeTree() \
ORDER BY (liker_id) \
POPULATE \
AS SELECT liker_id, first_name, countState() as cnt FROM \
( \
SELECT * FROM task.likes as l INNER JOIN task.user_profiles as p \
ON l.liker_id = p.id \
) \
GROUP BY liker_id, first_name;

In [ ]:
CREATE MATERIALIZED VIEW task.mv_likesProfiles \
ENGINE = AggregatingMergeTree() \
ORDER BY (liker_id) \
POPULATE \
AS SELECT liker_id, first_name, countState() as cnt FROM \
( \
SELECT * FROM task.likes as l INNER JOIN task.user_profiles as p \
ON l.liker_id = p.id \
) \
GROUP BY liker_id, first_name;

In [102]:
# materilized view is being created to dislay liker id , first name and the likes they made.
client1.execute("CREATE MATERIALIZED VIEW task.mv_likesProfiles ON CLUSTER kube_clickhouse_cluster \
ENGINE = AggregatingMergeTree() \
ORDER BY (liker_id) \
POPULATE \
AS SELECT liker_id, first_name, countState() as cnt FROM \
( \
SELECT * FROM task.likes as l INNER JOIN task.user_profiles as p \
ON l.liker_id = p.id \
) \
GROUP BY liker_id, first_name;")

ServerException: Code: 371.
DB::Exception: There was an error on [clickhouse-1:9000]: An error occurred before execution: Code: 371, e.displayText() = DB::Exception: There are two exactly the same ClickHouse instances clickhouse-1:9000 in cluster kube_clickhouse_cluster (version 20.5.2.7 (official build)). Stack trace:

0. Poco::Exception::Exception(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, int) @ 0x10ed0da0 in /usr/bin/clickhouse
1. DB::Exception::Exception(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, int) @ 0x95c923d in /usr/bin/clickhouse
2. DB::DDLQueryStatusInputStream::readImpl() @ 0xdb760ab in /usr/bin/clickhouse
3. DB::IBlockInputStream::read() @ 0xd999b1d in /usr/bin/clickhouse
4. DB::AsynchronousBlockInputStream::calculate() @ 0xd98f9a7 in /usr/bin/clickhouse
5. ? @ 0xd98fe08 in /usr/bin/clickhouse
6. ThreadPoolImpl<ThreadFromGlobalPool>::worker(std::__1::__list_iterator<Threa

In [ ]:
CREATE TABLE task.ddl_mv_likesProfiles AS task.mv_likesProfiles ENGINE = Distributed(kube_clickhouse_cluster, task, mv_likesProfiles);

In [73]:
client1.execute("CREATE TABLE task.ddl_mv_likesProfiles ON CLUSTER kube_clickhouse_cluster AS task.mv_likesProfiles ENGINE = Distributed(kube_clickhouse_cluster, task, mv_likesProfiles);")

[('clickhouse-2', 9000, 0, '', 2, 0),
 ('clickhouse-1', 9000, 0, '', 1, 0),
 ('clickhouse-3', 9000, 0, '', 0, 0)]

In [230]:
client1.execute("SELECT owner_id, count(post_id) as cnt FROM \
task.ddl_posts WHERE length(text) > 255 GROUP BY owner_id order by cnt desc limit 5;")

[('-41939351', 43189),
 ('-36151406', 40372),
 ('-84018481', 17754),
 ('-36025237', 12609),
 ('-35243698', 9821)]

In [75]:
client1.execute("SELECT table, formatReadableSize(sum(data_compressed_bytes)) FROM system.parts WHERE database='task' AND active=1 GROUP BY database, table;")

[('.inner.mv_likesProfiles', '39.17 KiB'),
 ('.inner.mv1_posts', '779.40 KiB'),
 ('.inner.mv2_posts', '344.19 KiB'),
 ('likes', '52.95 MiB'),
 ('.inner.mv_likes_profiles', '142.80 KiB'),
 ('.inner.mv2_user_profiles', '895.62 KiB'),
 ('.inner.mv1_followers', '6.82 MiB'),
 ('.inner.dist_mv_likesProfiles', '142.80 KiB'),
 ('.inner.mv1_likes', '460.83 KiB'),
 ('friends', '159.56 MiB'),
 ('.inner.mv2_likes', '406.12 KiB'),
 ('followers', '170.58 MiB'),
 ('.inner.mv1_friends', '2.32 MiB'),
 ('posts', '118.81 MiB'),
 ('user_profiles', '43.05 MiB'),
 ('.inner.mv1_user_profiles', '5.98 KiB')]